In [1]:
from arcgis.gis import GIS
gis=GIS("home")

In [2]:
import arcpy
import requests
from zipfile import ZipFile
import io
import os

In [3]:
prism_zip_url = "https://ftp.prism.oregonstate.edu/normals/monthly/4km/ppt/PRISM_ppt_30yr_normal_4kmM4_all_bil.zip"
prism_data_zip = requests.get(prism_zip_url, allow_redirects=True)

open(r"C:\Users\rockx057\Documents\ArcGIS\Projects\Lab 2 precip cube\prism_zip_data.zip", "wb").write(prism_data_zip.content)
     


27839341

In [4]:
with ZipFile(r"C:\Users\rockx057\Documents\ArcGIS\Projects\Lab 2 precip cube\prism_zip_data.zip", 'r') as prism_data:
    prism_data.extractall(
        path=r"C:\Users\rockx057\Documents\ArcGIS\Projects\Lab 2 precip cube")

In [10]:
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\rockx057\Documents\ArcGIS\Projects\Lab 2 precip cube\Default.gdb", workspace=r"C:\Users\rockx057\Documents\ArcGIS\Projects\Lab 2 precip cube\Default.gdb"):
    arcpy.management.CreateMosaicDataset(
        in_workspace=r"C:\Users\rockx057\Documents\ArcGIS\Projects\Lab 2 precip cube\Default.gdb",
        in_mosaicdataset_name="Prism_data_mosaic3",
        coordinate_system='GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
    )

In [11]:
arcpy.management.AddRastersToMosaicDataset(
    in_mosaic_dataset="Prism_data_mosaic3",
    raster_type="Raster Dataset",
    input_path=r"C:\Users\rockx057\Documents\ArcGIS\Projects\Lab 2 precip cube\BILs",
)

<Result 'Prism_data_mosaic3'>

In [12]:
arcpy.management.CalculateField(
    in_table=r"Prism_data_mosaic3\Footprint",
    field="Variable",
    expression= '"30Yr Normal"',
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'Prism_data_mosaic3\\Footprint'>

In [18]:
arcpy.management.CalculateField(
    in_table="Prism_data_mosaic3\Footprint",
    field="Timestamp",
    expression="DateAdd(Date(1993,0,1), $feature.OBJECTID-1, 'year')",
    expression_type="ARCADE",
    code_block="",
    field_type="DATE",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'Prism_data_mosaic3\\Footprint'>

In [16]:
arcpy.md.BuildMultidimensionalInfo(
    in_mosaic_dataset=r"C:\Users\rockx057\Documents\ArcGIS\Projects\Lab 2 precip cube\Default.gdb\Prism_data_mosaic3",
    variable_field="Variable",
    dimension_fields="Timestamp # #",
    variable_desc_units=None,
    delete_multidimensional_info="NO_DELETE_MULTIDIMENSIONAL_INFO"
)

<Result 'C:\\Users\\rockx057\\Documents\\ArcGIS\\Projects\\Lab 2 precip cube\\Default.gdb\\Prism_data_mosaic3'>

In [17]:
arcpy.md.MakeMultidimensionalRasterLayer(
    in_multidimensional_raster="Prism_data_mosaic3",
    out_multidimensional_raster_layer="Prism_data_mosaic_MultidimLayer2",
    variables="'30Yr Normal'",
    dimension_def="ALL",
    dimension_ranges=None,
    dimension_values=None,
    dimension="",
    start_of_first_iteration="",
    end_of_first_iteration="",
    iteration_step=None,
    iteration_unit="",
    template='-125.020833333333 24.0624999997935 -66.4791666661985 49.9375000000005 GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
    dimensionless="DIMENSIONS",
    spatial_reference=None
)

<Result 'Prism_data_mosaic_MultidimLayer2'>

In [19]:
arcpy.stpm.CreateSpaceTimeCubeMDRasterLayer(
    in_md_raster="Prism_data_mosaic_MultidimLayer2",
    output_cube=r"C:\Users\rockx057\Documents\ArcGIS\Projects\Lab 2 precip cube\prism_cube.nc",
    fill_empty_bins="ZEROS"
)

<Result 'C:\\Users\\rockx057\\Documents\\ArcGIS\\Projects\\Lab 2 precip cube\\prism_cube.nc'>